In [ ]:
%%capture
# Cài đặt Unsloth và Transformers
!pip install pip3-autoremove
!pip install torch torchvision torchaudio xformers --index-url https://download.pytorch.org/whl/cu128
!pip install unsloth
!pip install transformers==4.55.4 # Phải khớp lúc train
!pip install --no-deps trl==0.22.2
!pip install sacrebleu

In [ ]:
import os
import torch
import sacrebleu
import numpy as np
import unicodedata
import re
import html
from unsloth import FastLanguageModel
from datasets import Dataset
from trl import SFTTrainer, SFTConfig

# --- CẤU HÌNH ĐƯỜNG DẪN & THAM SỐ CUỐI CÙNG ---
max_seq_length = 512 # Giữ nguyên lúc train
dtype = None
load_in_4bit = True

TEST_EN_PATH = "/kaggle/input/vlsp-medical/MedicalDataset_VLSP/public_test.en.txt"
TEST_VI_PATH = "/kaggle/input/vlsp-medical/MedicalDataset_VLSP/public_test.vi.txt"
CHECKPOINT_PATH = "/kaggle/input/cp-vi2en/outputs-phase2/checkpoint-2000" # Checkpoint cuối cùng

# KẾT LUẬN TỪ QUÁ TRÌNH PHÂN TÍCH (571 tokens + 10 buffer)
MAX_SAFE_NEW_TOKENS = 581
BATCH_SIZE = 32 # Tối ưu hóa tốc độ Inference (Có thể thử 64 nếu T4 cho phép)

In [ ]:
#--- 1. LOAD MODEL VÀ ADAPTERS ---
print(f"Đang tải model từ: {CHECKPOINT_PATH}...")
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = CHECKPOINT_PATH, 
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    device_map = "cuda:0",
)
FastLanguageModel.for_inference(model)
print("Model đã sẵn sàng cho Inference!")

In [ ]:
# --- 2. LOAD VÀ LÀM SẠCH DATA ---
def preprocess_text(text):
    if not isinstance(text, str): return ""
    text = html.unescape(text)
    text = re.sub(r'<[^>]+>', '', text)
    text = re.sub(r'(?:https?://|www\.)\S+', '', text)
    text = re.sub(r'[\x00-\x1F\x7F-\x9F]', '', text)
    text = unicodedata.normalize('NFC', text)
    text = re.sub(r'\s+', ' ', text).strip()
    return text

print(" Đang load và làm sạch Public Test...")
with open(TEST_EN_PATH, 'r', encoding='utf-8') as f: en_data = [preprocess_text(l) for l in f]
with open(TEST_VI_PATH, 'r', encoding='utf-8') as f: vi_data = [preprocess_text(l) for l in f]

# Reference phải là list of list cho sacrebleu
en_references = [[l.strip()] for l in en_data]
vi_references = [[l.strip()] for l in vi_data]
print(f" Đã load {len(en_data)} câu Test (sẵn sàng tính toán).")

In [ ]:
#  PHASE 4: TÍNH BLEU SCORE CUỐI CÙNG (FINAL VERSION)
# ==============================================================================
import sacrebleu
import torch

# sys_prompt_en_vi = (
#     "Bạn là một biên dịch viên y tế chuyên nghiệp. "
#     "Nhiệm vụ của bạn là dịch chính xác văn bản y khoa từ tiếng Anh sang tiếng Việt, "
#     "đảm bảo văn phong khoa học và thuật ngữ chính xác."
# )
sys_prompt_vi_en = (
    "You are a professional medical translator. "
    "Your task is to accurately translate the following Vietnamese medical text into English. "
    "Ensure correct medical terminology and academic style."
)

# 1. QUAN TRỌNG NHẤT: Đổi padding sang trái cho tác vụ sinh văn bản
tokenizer.padding_side = "left" 

def batch_translate_and_score(source_texts, target_references, direction, batch_size=BATCH_SIZE):
    sys_prompt = sys_prompt_vi_en
    hypotheses = []
    model.eval()
    
    print(f" Bắt đầu dịch {direction} (Batch size: {batch_size})...")
    
    # Duyệt qua từng batch (Ví dụ: 16 câu một lần)
    for i in range(0, len(source_texts), batch_size):
        batch = source_texts[i : i + batch_size]
        
        # 2. Tạo Prompt và Tokenize ngay trong vòng lặp (Tiết kiệm RAM)
        batch_prompts = []
        for text in batch:
            messages = [{"role": "system", "content": sys_prompt}, {"role": "user", "content": text}]
            prompt_str = tokenizer.apply_chat_template(
                messages,
                tokenize=False,
                add_generation_prompt=True,
                enable_thinking=False
            )
            batch_prompts.append(prompt_str)
            
        # 3. Tokenize batch này và đẩy lên GPU ngay (Vì batch nhỏ nên an toàn)
        inputs = tokenizer(
            batch_prompts,
            return_tensors="pt",
            padding=True,
            truncation=True,
            max_length=max_seq_length,
        ).to("cuda")

        # Dynamic max tokens: Giới hạn độ dài sinh ra để không bị cụt hoặc quá dài
        src_lens = inputs.attention_mask.sum(dim=1)
        max_src_len = int(src_lens.max())
        DYNAMIC_MAX_NEW_TOKENS = min(int(max_src_len * 1.5) + 20, 512) 

        # 4. Sinh văn bản
        with torch.no_grad():
            outputs = model.generate(
                input_ids = inputs.input_ids,
                attention_mask = inputs.attention_mask,
                max_new_tokens = DYNAMIC_MAX_NEW_TOKENS,
                do_sample = False,      # Greedy search (tốt nhất cho đánh giá)
                num_beams = 1,
                use_cache = True,
                pad_token_id = tokenizer.pad_token_id,
                eos_token_id = tokenizer.eos_token_id 
            )
        
        # 5. Cắt bỏ phần prompt khỏi kết quả (Slicing chuẩn xác)
        # Lấy độ dài thực tế của input (bao gồm padding bên trái)
        input_len_total = inputs.input_ids.shape[1]
        
        for output_seq in outputs:
            generated_tokens = output_seq[input_len_total:] # Cắt sạch phần prompt
            decoded = tokenizer.decode(generated_tokens, skip_special_tokens=True).strip()
            hypotheses.append(decoded)
            
        print(f"   -> Đã dịch {direction}: {min(i + batch_size, len(source_texts))}/{len(source_texts)} câu...", end='\r')
        
    # 6. Tính BLEU (Format chuẩn cho sacrebleu: list of lists)
    # Target references phải được bọc trong 1 list nữa
    bleu = sacrebleu.corpus_bleu(hypotheses, [target_references])
    
    return bleu.score, hypotheses


print("\n" + "="*60)
print(f"BẮT ĐẦU TÍNH BLEU SCORE (VI-EN SPECIALIST)")
print("="*60)

# Chạy test
# valid_en: List các câu tiếng Anh
# vi_references: List các câu đáp án tiếng Việt (dạng string thuần túy: ['câu 1', 'câu 2'])
bleu_vi_en, hypotheses_vi_en = batch_translate_and_score(vi_data, en_data, "vi_en", batch_size=BATCH_SIZE)
print(f"\n\n KẾT QUẢ BLEU SCORE (VIỆT -> ANH): {bleu_vi_en:.2f}")